[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_)


In [1]:
from pyfian.time_value.present_value import present_value_annuity
present_value_annuity(100, rate=0.1, periods=5)

from pyfian.time_value.present_value import present_value_growing_annuity
present_value_growing_annuity(100, growth=0.05, rate=0.1, periods=5)



415.0591272329889